<a href="https://colab.research.google.com/github/bnriiitb/whisper-fine-tuning/blob/main/model_evaluation_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [86]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4
!apt update
!apt install -y ffmpeg
!apt-get install libsndfile1

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/jonathonf

In [87]:
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-1vr1y432
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-1vr1y432
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget https://storage.googleapis.com/test_public_bucket/labelled/Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip
!unzip Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip

--2022-12-04 17:59:50--  https://storage.googleapis.com/test_public_bucket/labelled/Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.202.128, 74.125.199.128, 74.125.20.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 90658931 (86M) [application/zip]
Saving to: ‘Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip.1’

Telugu_Chai_Bisket_ 100%[===================>]  86.46M  90.5MB/s    in 1.0s    

2022-12-04 17:59:51 (90.5 MB/s) - ‘Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip.1’ saved [90658931/90658931]

Archive:  Telugu_Chai_Bisket_Stories_16-08-2021_14-17.zip
replace Chai_Bisket_Stories_16-08-2021_14-17/3_1_481file-idc4p8tJhhC0o.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import datasets
from datasets import load_dataset, DatasetDict, Dataset, Audio

ds_1 = load_dataset("/content/Chai_Bisket_Stories_16-08-2021_14-17")

ds_1 = ds_1['train']
print(ds_1)

ds_2 = load_dataset("json",data_files="/content/Chai_Bisket_Stories_16-08-2021_14-17/data.json")
ds_2 = ds_2['train']
print(ds_2)

final_dataset = datasets.concatenate_datasets([ds_1, ds_2], axis=1)
print(final_dataset)

train_test_dataset = final_dataset.train_test_split(test_size=0.30)
print(train_test_dataset)


train_test_dataset = train_test_dataset.remove_columns(['audioFilename','collectionSource', 'snr', 'duration', 'gender',])
print(train_test_dataset)

In [ ]:
# from transformers import WhisperFeatureExtractor
# from transformers import WhisperTokenizer
# from transformers import WhisperProcessor
# from datasets import Audio

# feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")
# tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Telugu", task="transcribe")
# processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Telugu", task="transcribe")

# common_voice = train_test_dataset.cast_column("audio", Audio(sampling_rate=16000))

In [ ]:
# def prepare_dataset(batch):
#     # load and resample audio data from 48 to 16kHz
#     audio = batch["audio"]

#     # compute log-Mel input features from input audio array 
#     batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

#     # encode target text to label ids 
#     batch["labels"] = tokenizer(batch["text"]).input_ids
#     # batch["labels"] = tokenizer(batch["sentence"]).input_ids
#     return batch

# train_test_dataset = train_test_dataset.map(prepare_dataset, remove_columns=train_test_dataset.column_names["train"], num_proc=8)    

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="bnriiitb/whisper-small-te")  # change to "your-username/the-name-you-picked"

# def transcribe(audio):
#     text = pipe(audio)["text"]
#     return text

# iface = gr.Interface(
#     fn=transcribe, 
#     inputs=gr.Audio(source="microphone", type="filepath"), 
#     outputs="text",
#     title="Whisper Small Telugu",
#     description="Realtime demo for Telugu speech recognition using a fine-tuned Whisper small model.",
# )

# iface.launch()

In [ ]:
# dataset = load_dataset("/content/Chai_Bisket_Stories_16-08-2021_14-17")
dataset = load_dataset("json",data_files="/content/Chai_Bisket_Stories_16-08-2021_14-17/data.json")
dataset

In [ ]:
from IPython.display import Audio
file_name = '7_381file-id_j-j_NR0T1Y.wav'
Audio(f"/content/Chai_Bisket_Stories_16-08-2021_14-17/{file_name}")

In [ ]:
dataset['train'][66]

In [ ]:
train_test_dataset['train'][20]

In [ ]:
Audio(train_test_dataset['train'][20]['audio']['path'])

In [ ]:
pipe(f"/content/Chai_Bisket_Stories_16-08-2021_14-17/{file_name}",language='Telugu',device=2,task='transcribe')

In [ ]:
!pip install git+https://github.com/openai/whisper.git -q
!pip install -U tqdm

In [ ]:
import whisper
whisper_small_model = whisper.load_model("small")

In [ ]:
from IPython.display import Audio

info = dataset['train'][156]
print(info['text'])
file_name = info['audioFilename']
Audio(f"/content/Chai_Bisket_Stories_16-08-2021_14-17/{file_name}")

In [ ]:
whisper_small_model.transcribe(f"/content/Chai_Bisket_Stories_16-08-2021_14-17/{file_name}",language="Telugu")